In [1]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
import os
import json
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
device = "cuda"

/home/alsdyd98/anaconda3/envs/instblip/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# https://github.com/salesforce/LAVIS/tree/main/projects/instructblip#prepare-vicuna-weights
#!pip3 install "fschat[model_worker,webui]"
#!python3 -m fastchat.serve.cli --model-path lmsys/vicuna-7b-v1.5

In [2]:
from lavis.models import load_model_and_preprocess
# loads InstructBLIP model
model, vis_processors, _ = load_model_and_preprocess(name="blip2_vicuna_instruct", model_type="vicuna7b", is_eval=True, device=device)

/home/alsdyd98/anaconda3/envs/instblip/lib/python3.9/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': './llm/vicuna-7b'. Use `repo_type` argument if needed.

In [ ]:
class AnswerGenerator:
    def __init__(self, model, preprocessor, device='cuda'):
        self.model = model
        self.preprocessor = preprocessor
        self.device = device

    @torch.no_grad()
    def _load(self, image):
        if type(image) == str:
            raw_image = Image.open(image).convert("RGB")
        else:
            raw_image = image
        image_emb = self.preprocessor["eval"](raw_image).unsqueeze(0).to(device)
        return image_emb

    def inference(self, image, prompt):
        img_emb = self._load(image)
        qaSet = {"image": img_emb, "prompt": prompt}
        return self.model.generate(qaSet)

In [ ]:
llm = AnswerGenerator(model, vis_processors, device)
base_dir = "."
# key: imageFile_name, value: [{key(Q_i):value(question description)}]
with open(f'{base_dir}/qSet.json', 'r') as f:
    question_dict = json.load(f)

answer_dict = {}
for fileName in question_dict.keys():
    answer_dict[fileName] = {}
    for q in question_dict[fileName]:
        img_dir = f'{base_dir}/images/fileName'
        ans = llm.inference(img_dir, question_dict[fileName][q])
        ansKey = "A" + q[1:]
        answer_dict[fileName][ansKey] = ans

with open(f'{base_dir}/answer_set.json', 'w') as outfile:
    json.dump(answer_dict, outfile)